BOW : 단어의 순서 고려 X , 단어의 빈도수를 기반으로 만든 행렬  
1) 단어 추출 -> 단어에 대해 정수 인덱스 설정  
2) 단어의 등장 횟수를 저장한 벡터를 생성  

In [1]:
text = '정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.'

In [2]:
# 퀴즈 1. text 에서 .(마침점) 을 제거

text=text.replace('.','')
text

'정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다'

In [3]:
from konlpy.tag import Okt
okt = Okt()

In [5]:
token = okt.morphs(text)
token

['정부', '가', '발표', '하는', '물가상승률', '과', '소비자', '가', '느끼는', '물가상승률', '은', '다르다']

In [20]:
word2index = {} # { 단어:인덱스번호 }
bow=[] # [단어빈도수 , 단어빈도수 , ]

In [21]:
for v in token :
    if v not in word2index.keys():
        word2index[v]=len(word2index)
        bow.insert(len(word2index)-1,0)
    
    bow[word2index[v]]+=1
        
word2index

{'정부': 0,
 '가': 1,
 '발표': 2,
 '하는': 3,
 '물가상승률': 4,
 '과': 5,
 '소비자': 6,
 '느끼는': 7,
 '은': 8,
 '다르다': 9}

In [22]:
bow

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

전체문서 (n개 문서) = {d1,d2,...,dn}  
단어집합 (m개 단어) = {t1,t2,...,tm}  
Xij = 문서 di 에 있는 단어 tj의 출현 빈도수  

In [23]:
# 전부 다 BOW 생성 모듈들
from sklearn.feature_extraction import DictVectorizer , dict_vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer , HashingVectorizer

C:\Users\i\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_extraction.dict_vectorizer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction. Anything that cannot be imported from sklearn.feature_extraction is now part of the private API.
  warnings.warn(message, FutureWarning)


__DictVectorizer__

In [24]:
v = DictVectorizer(sparse=False) # sparse : 비어있는 정도로 이해. True 시 정보 압축이 없고 메모리 차지 큼
d = [{'a:':1 ,'b':2},{'b':3 , 'c':1}] # [문서1 , 문서2] 로 가정 a,b,c는 각 단어 

In [25]:
v.fit_transform(d) # bow 생성

array([[1., 2., 0.],
       [0., 3., 1.]])

In [26]:
v.feature_names_

['a:', 'b', 'c']

In [27]:
v.transform({'c':5,'d':2})

array([[0., 0., 5.]])

__CountVectorizer__  
1. 문서 -> 토큰 변환  
2. 토큰의 빈도수 조사  
3. bow 벡타 생성  

In [28]:
corpus = ['This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?']

In [31]:
cvt = CountVectorizer()
cvt.fit(corpus)

CountVectorizer()

In [32]:
cvt.vocabulary_ # 알파벳 순으로 인덱스를 매긴 듯 하다.

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [33]:
cvt.transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

In [36]:
# 새로운 문서가 입력 -> bow 변환
new_text = ['This is the second document.']
cvt.transform(new_text).toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [38]:
cvt = CountVectorizer(stop_words=['and','is','the','this'])
cvt.fit(corpus)
cvt.vocabulary_

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [39]:
cvt = CountVectorizer(analyzer='char')
cvt.fit(corpus)
cvt.vocabulary_

{'t': 16,
 'h': 8,
 'i': 9,
 's': 15,
 ' ': 0,
 'e': 6,
 'f': 7,
 'r': 14,
 'd': 5,
 'o': 13,
 'c': 4,
 'u': 17,
 'm': 11,
 'n': 12,
 '.': 1,
 'a': 3,
 '?': 2,
 'l': 10}

In [40]:
cvt = CountVectorizer(token_pattern = 't[a-zA-z]+')
cvt.fit(corpus)
cvt.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [41]:
cvt = CountVectorizer(ngram_range=(2,2))
cvt.fit(corpus)
cvt.vocabulary_

{'this is': 12,
 'is the': 2,
 'the first': 7,
 'first document': 1,
 'the second': 9,
 'second second': 6,
 'second document': 5,
 'and the': 0,
 'the third': 10,
 'third one': 11,
 'is this': 3,
 'this the': 13,
 'the last': 8,
 'last document': 4}

In [42]:
cvt = CountVectorizer(ngram_range=(1,2),token_pattern='t\w+')
cvt.fit(corpus)
cvt.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

## __TF-IDF__  
[https://en.wikipedia.org/wiki/Tf%E2%80%93idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

- TF : term frequency , 해당 문서에서 각 단어가 얼마나 등장했는가 (dtm matrix)  
- DF : document frequency , 해당 단어가 전체 문서중 몇개 문서에 등장하였는가  
- IDF : Inverse document frequency : DF 의 역수로 단어의 중요도 역할을 하는 수치, 출현문서가 높을수록 중요도가 낮아질 수 있게 한다.  
- TF-IDF : 각 단어에 대하여 문서에서 빈도수를 조사하는데 , IDF를 같이 고려하여 전체 문서 기준 조금 더 신뢰있는 빈도수치로 각 단어를 조사할 수 있다. 단어의 중요도 (Weighting) 계산하는데에 많이 쓰인다.  

__함수로 표현한다면__ 
TF : tf(d,t) = d문서에서 단어t는 몇번이나 등장하는가  
DF : df(t) = 단어 t가 등장하는 문서의 수는 몇개인가 (등장횟수는 상관없이 YES or NO 로만)  
IDF : log(n / (1+df(t))) = 전체 문서의 수(n)를 df+1(스무딩) 로 나눈 수치의 로그 스케일 값  
DF<->IDF : 반비례 관계임은 맞지만 n의 선형적 증가의 효과를 줄이기 위해 로그 스케일을 택하며 zero-division 방지를 위해 스무딩 작업 또한 진행한다.  


In [67]:
docs =[
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]

In [68]:
# TF IDF 계산 잠깨기

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
DTM = cv.fit_transform(docs).toarray()
DTM_df = pd.DataFrame(DTM,columns=cv.vocabulary_.keys())
DTM_df

,먹고,싶은,사과,바나나,길고,노란,저는,과일이,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [73]:
def get_idf(col):
    L = len(col)
    not_in = (col==0).sum()
    return np.log(L/(L-not_in+1))

IDF = DTM_df.apply(get_idf)
IDF

먹고     0.693147
싶은     0.693147
사과     0.693147
바나나    0.287682
길고     0.287682
노란     0.693147
저는     0.287682
과일이    0.693147
좋아요    0.693147
dtype: float64

In [71]:
def get_tfidf(col):
    col = col*IDF
    return col

TFIDF = DTM_df.apply(get_tfidf,axis=1)
TFIDF

,먹고,싶은,사과,바나나,길고,노란,저는,과일이,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147
